# Adding Back the Tides - CIOPS
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

#### unlike what we did with SSC in this file make all add back tides days seperate to reduce space requirement

In [53]:
startday = [dt.datetime(2015,11,22)+dt.timedelta(days=i) for i in range(int(1498))]
print(startday[-1])

2019-12-28 00:00:00


In [52]:
for i in range(131,len(startday)-1):
    print(i)

131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380


In [68]:
folders = [dt.datetime(2015,11,22)+dt.timedelta(days=7*i) for i in range(int(214))]
print(folders[-1])
folders = np.repeat(folders,7)

2019-12-22 00:00:00


In [69]:
np.shape(startday)

(1498,)

In [70]:
np.shape(folders)

(1498,)

In [76]:
# start = dt.datetime(2015,11,22) #start day of your run

#dates for each run
# numdays = 2 #15 for all except the last run (7) 
# date_list = [start + dt.timedelta(days=x) for x in range(numdays)]
# folderday = [start+ dt.timedelta(days=7),start+ dt.timedelta(days=7*2)]#,start+ dt.timedelta(days=7*3)]
# folderday = np.repeat(folderday,7)
# folderday = folderday[:-6]
date_list = [startday[13],startday[14]]
folderday = [folders[13], folders[14]]

In [77]:
date_list

[datetime.datetime(2015, 12, 5, 0, 0), datetime.datetime(2015, 12, 6, 0, 0)]

In [78]:
folderday

[datetime.datetime(2015, 11, 29, 0, 0), datetime.datetime(2015, 12, 6, 0, 0)]

In [79]:
"{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[0], date_list[0], date_list[0])

'2015112900/BC12_1h_grid_U_2D_20151205_20151205.nc'

In [80]:
path = Path("/ocean/mdunphy/CIOPSW-BC12/")

drop_vars = (
    "sbu", "tauuo", "time_counter_bounds", "time_instant_bounds", "uos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
ut_h = mydata['ubar']

IndexError: list index out of range

In [81]:
files

[[], []]

In [20]:
ut_h

<xarray.DataArray 'ubar' (time_counter: 48, y: 1020, x: 714)>
dask.array<concatenate, shape=(48, 1020, 714), dtype=float32, chunksize=(24, 1020, 714), chunktype=numpy.ndarray>
Coordinates:
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
  * time_counter  (time_counter) datetime64[ns] 2015-11-22T01:00:00 ... 2015-...
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
Dimensions without coordinates: y, x
Attributes:
    long_name:           ocean barotropic current along i-axis
    units:               m/s
    online_operation:    instant
    interval_operation:  60 s
    interval_write:      1 h
    cell_methods:        time: point (interval: 60 s)

In [21]:
drop_vars = (
    "sbv", "tauvo", "time_counter_bounds", "time_instant_bounds", "vos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_V_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
vt_h = mydata['vbar']

In [22]:
# 2D hourly outputs are alraedy in bartropic form so you dont need to do any conversion!

In [23]:
# Now get the required data from the daily files
drop_vars = (
    "depthu_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_U_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['uo']

drop_vars = (
    "depthv_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_V_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vo']

In [24]:
# we WILL need to do some conversions here, so get e3t from the mesh_mask file
mydata = xr.open_dataset("/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc")
e3t = mydata['e3t_0']
e3t

<xarray.DataArray 'e3t_0' (t: 1, z: 75, y: 1020, x: 714)>
[54621000 values with dtype=float64]
Dimensions without coordinates: t, z, y, x

In [25]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'z': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'z': 'depthv'})

In [26]:
#calcuate bartropic component
ut_d = (u_d*e3u[0,:,:,:]).sum(dim='depthu')/e3u[0,:,:,:].sum(dim='depthu')

In [27]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [28]:
offset = dt.timedelta(hours=-23) # daily avg timestamp as next day time 0, offset by -23 hours to line up with hourly data
uc_h_interp = uc_d.resample(time_counter="1H", loffset=offset).interpolate("linear")
#instead of taking 12hours off each side like when done with SSC, this method takes off 24hours on the end!

In [33]:
u_new = ut_h  + uc_h_interp

In [38]:
u_new = u_new.isel(time_counter = np.arange(0,24,1))

/home/rbeutel/.local/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [41]:
u_new = u_new.rename('vozocrtx')

In [42]:
u_new

<xarray.DataArray 'vozocrtx' (time_counter: 24, y: 1020, x: 714, depthu: 75)>
dask.array<getitem, shape=(24, 1020, 714, 75), dtype=float64, chunksize=(24, 1020, 714, 75), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2015-11-22T01:00:00 ... 2015-...
    nav_lat       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(1020, 714), meta=np.ndarray>
    time_instant  (time_counter) datetime64[ns] dask.array<chunksize=(24,), meta=np.ndarray>
  * depthu        (depthu) float32 0.5075 1.558 2.669 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x

In [ ]:
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))

In [ ]:
#calcuate bartropic component
vt_d = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

In [ ]:
#subtract from v to get baroclinic component
vc_d = v-vt_d 

In [ ]:
vc_d.load(scheduler="processes", num_workers=n)

In [ ]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [ ]:
v_new = vt_h  + vc_h_interp

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))